<a href="https://colab.research.google.com/github/averkief/My_Date_Analytics/blob/main/Popular_Russian_syllables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Предисловие***<br>
Когда мой сын подрастал и начинал познавать мир, то дошло дело и до чтения.
Современные методики рекомендуют учить детей не буквам, а слогам. Указывая на то, что так будет проще научиться читать.<br>
И так в русском языке 10 гласных букв и 20 согласных. Отбросив соединения согласных с гласными которые не соответсвуют правилам русского языка, получаем 158 правильных слогов.<br>
И тут у меня возник вопрос, а какие слоги самые популярные, какие лучше выучить первыми???<br><br>
ИТАК ЗАДАЧА: Какой слог встречается чаще всего? И бонусом, какая длина чаще всего у этих слов?

**Последовательность действий**<br>
*Этап 1: Подключение к источнику и получение текста книги*<br>
Выбор книги выпал на "Война и Мир" Л.Н.Толстова, как самая большая по объему. Текст книги есть в интернете. Требуется спарсить текст с сайта<br>
*Этап 2: Получение текста, его обрабока и перевод в строку* <br>
Разбиваем текст на слова и извлекаем пунктуацию, предлоги, местоимения слова не относящиеся к словам со слогами<br>
*Этап 3: Внесение данных для нахождения слогов* <br>
Подготовка искомых слогов и счетчиков подсчета<br>
*Этап 4: Заготовка Дата-Фрейма* <br>
Создаем основу ДатаФрема с нулевыми значениями<br>
*Этап 5: Обрабока слов и учет количество слогов* <br>
Обработка каждого слова<br>
Ссылка на dataframe csv:<br>
https://drive.google.com/drive/folders/198CcQ6XPT9pYd3yzQ5QDDwN5IMZRp1pf?usp=sharing<br>
*Этап 6: Анализ полученных данных* <br>
Получение информации: количественное значиени, графики<br>

### Подключение библиотек и глобальных переменных

In [1]:
import requests # Бибилиотека запросов к web-сервисам
from bs4 import BeautifulSoup as bs # Библиотека парсинга web-страниц
import pandas as pd # Библиотека для работы с таблицами
import numpy as np

# Для обработки текста
import nltk # Библиотека для работы с текстом
from nltk.tokenize import word_tokenize # токенайзер - разбивает непрерывный текст на токены(слова). word_tokenize - для разделения слов на токены.
#nltk.download('all')
nltk.download('punkt')
#nltk.download('stopwords')

import string # используем для получения знаков пунктуации

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Глобальные переменные
URL = 'https://ilibrary.ru' # + index.html - Страница оглавления книг "Война и Мир"
URL_CHAPTER  = '/text/11/index.html' # страница оглавления
URL_ENDINGS = [] # Список окончаний url страниц книги
TEXT_BOOK = '' # Полный текст (все слова больше 2 букв) для анализа слогов
TOTAL_SLOG = [] # Основа датафрейма из всех возможные варианты слогов, даже те которые противоречат поравилам русского языка
vowels = 'ауоыиэяюёе' # гласные буквы
consonants = 'бвгджзклмнпрстфхцчшщ' # согласные буквы

### Этап 1: Подключение к источнику

In [3]:
# Подключение к странице оглавления
connect_url = requests.get(URL + URL_CHAPTER)
if connect_url.status_code == 200:
  text_in_page = connect_url.text
else:
  print('СТРАНИЦА НЕ ДОСТУПНА')

In [4]:
# Функция проверка существующей ссылки
def check_href(href_url):
  ''' Проверка значение с пердидущим значением списка'''
  #global URL_ENDINGS

  if len(URL_ENDINGS) < 1:
    URL_ENDINGS.append(href_url)
  else:
    if href_url != URL_ENDINGS[-1]:
      URL_ENDINGS.append(href_url)

In [5]:
# Получение всех страниц книги
chapter_page = bs(text_in_page, 'html.parser')
href_to_url = chapter_page.find_all('p')
for href_one in href_to_url:
  check_href(href_one.a['href'])

# Проверка количества стра быниц в списке, должноть 361
print(len(URL_ENDINGS))   # содержание ['/text/11/p.1/index.html', '/text/11/p.2/index.html', '/text/11/p.3/index.html',...,'/text/11/p.361/index.html']

361


#### Этап 2: Получение текста, его обрабока и перевод в строку

In [6]:
# Списки пунктуации и стоп-слов (предлоги, местоимения и т.п.)
my_stop_punct = list(string.punctuation)
my_stop_punct = set(my_stop_punct + ['—'] + ['..'] + ['...'] + ['«'] + ['»'])
my_stop_not_words = {'abc', 'abcdefghiklmnopqrstuvwxyz', '123456789102030405060708090100110120130140150160'} # несуществующие слова, возможно защита сайта от копирования текста, найдены эмпирически
my_stop_words = {'а',
                 'без', 'бы', 'будто', 'был', 'была', 'было', 'более',
                 'в', 'во', 'вы', 'все', 'всю', 'вас', 'вам', 'всех',
                 'где', 'го',
                 'да', 'до', 'для',
                 'его', 'ее', 'её', 'еще', 'ещё', 'ему', 'ей', 'есть',
                 'ж', 'же',
                 'за', 'зачем',
                 'и', 'их', 'из', 'или', 'им',
                 'к', 'ко', 'как', 'кто', 'какой', 'какая', 'куда',
                 'ли',
                 'мы', 'мне', 'меня', 'моя', 'мой',
                 'не', 'ни', 'на', 'но', 'ну', 'нет', 'него', 'ней', 'нее', 'них', 'нами', 'нас', 'над', 'ним', 'нибудь',
                 'о', 'он', 'она', 'они', 'от', 'об',
                 'по', 'при', 'под', 'потому', 'после', 'про',
                 'раз', 'разве',
                 'с', 'со', 'сам', 'себе', 'свою',
                 'то', 'ты', 'та', 'те', 'так', 'тот', 'тем', 'там', 'тут', 'тебя', 'тоже', 'того', 'том', 'такой',
                 'у', 'уж', 'уже',
                 'хоть',
                 'что', 'чем', 'чтоб', 'чего', 'чуть', 'чтобы',
                 'это', 'эти', 'эту', 'этот', 'этого', 'этой', 'этом',
                 'я'}

In [7]:
def text_processing(text):
  ''' Принимает сырой текст, фильтрует его от знаков пунктуации, несуществущих слов и переводит в нижний регистр'''

  words = word_tokenize(text.lower()) # Разбмваем на список слов (токены)
  words = [word for word in words if word not in my_stop_punct] # Удаляем знаки пунктуации
  words = [word for word in words if word not in my_stop_not_words] # Удаляем стопслова

  all_words = ';'.join(words)
  return all_words

In [8]:
%%time
# Получение текта книги с каждой страницы
#for URL_ENDING in URL_ENDINGS:
for URL_ENDING in URL_ENDINGS[248:250]: # для тестов выберем несколько страниц
  print(URL + URL_ENDING) # Получаем ссылку к которой обратились за текстом книги
  connect_url_page = requests.get(URL + URL_ENDING)
  if connect_url_page.status_code == 200:
    text_page = connect_url_page.text
    one_page = bs(text_page, 'html.parser')
    text_in_url = one_page.find_all('span', class_='p')
    for text_one in text_in_url:
      TEXT_BOOK = TEXT_BOOK + text_processing(text_one.text) + ';' # Текст в функцию передается построчно до красной строки, так постепенно каждым абзацем наполняем полный текст

  else:
    print('ALARM!!! Ошибка подключения к странице сайта')

https://ilibrary.ru/text/11/p.249/index.html
https://ilibrary.ru/text/11/p.250/index.html
CPU times: user 569 ms, sys: 20.7 ms, total: 589 ms
Wall time: 3.35 s


In [9]:
list_TEXT_BOOK = TEXT_BOOK.split(';') # переведем в список для обрабоки
quantity_words_in_text = len(list_TEXT_BOOK) # Сколько всего обрабатывается слов
print(f'Количество слов в тексте: {quantity_words_in_text}')

Количество слов в тексте: 3995


In [10]:
#Определить самое длинное слово
max_word_in_text = max(list_TEXT_BOOK, key=len)
max_len_word_in_text = len(max_word_in_text)
# добавить суда проверку дефиса
print(f'Самое длинное слово из полного текста: "{max_word_in_text}", состоит из: {max_len_word_in_text} букв')

Самое длинное слово из полного текста: "правителю-администратору", состоит из: 24 букв


In [ ]:
# ПРОВЕРКА длинных слов
for long_word in list_TEXT_BOOK:
  if len(long_word) > 15:
    print(long_word)

In [ ]:
# ПРОВЕРКА слов на наличие дефисов
pp = 0
for dash_word in list_TEXT_BOOK:
  if '-' in dash_word:
    print(dash_word)
    pp += 1
print(pp)

In [ ]:
# ПРОВЕРКА слов на наличие символов
ppp = 0
for dash_word in list_TEXT_BOOK:
  for symbol in my_stop_punct:
    if symbol in dash_word:
      print(dash_word)
      ppp += 1
print(ppp)

### Этап 3: Внесение данных для нахождения слогов

In [12]:
# Получим возможные слоги
for i in range(2):     # Костыль, чтоб создать 2 раза список из слогов, для разделения обработки полного текста и сокращенного
  for consonant in consonants:
    for vowel in vowels:
      TOTAL_SLOG.append([consonant + vowel]) # Создали список с вложенным (2 уровня) списком слогов.
#Вывод [['ба'], ['бу'], ['бо'], ['бы'], ['би'], ['бэ'], ['бя'], ['бю'], ... , ['щё'], ['ще']]

In [14]:
len_TOTAL_SLOG = len(TOTAL_SLOG)

for quan_slog in range(len_TOTAL_SLOG):
  if quan_slog < (len_TOTAL_SLOG / 2):
    TOTAL_SLOG[quan_slog].insert(0, 'Полный') # Дополняем вложенный список типом текста Первую половину слогов
  else:
    TOTAL_SLOG[quan_slog].insert(0, 'Сокращенный') # Дополняем вложенный список типом текста Вторую половину слогов
  TOTAL_SLOG[quan_slog].append(0) # Дополняем вложенный список нулевыми значениями совпадений слогов

#Вывод [['Полный', 'ба', 0], ['Полный', 'бу', 0], ['Полный', 'бо', 0], ['Полный', 'бы', 0], ... , ['Сокращенный', 'щё', 0], ['Сокращенный', 'ще', 0]]

### Этап 4: Заготовка Дата-Фрейма

In [15]:
DF_SLOG = pd.DataFrame(TOTAL_SLOG, columns=['Текст', 'Слог', 'Частота']) # Сформируем предварительный датафрейм, 0 - тип текста, 1 - слог, 2 - количество совпадений

In [16]:
#Добавляем колонки справа столько сколько возможно максимальное слово, т.е. от 2 до 28. Заполняем их 0, чтоб потом вести счетчик подсчета
for num in range(2, max_len_word_in_text + 1):
  DF_SLOG[f'{num} буквы'] = 0

In [17]:
DF_SLOG.shape

(400, 26)

In [ ]:
DF_SLOG # Заготовка ДатаАрейма готова

### Этап 5: Обрабока слов и учет количество слогов

In [38]:
def word_processing(current_word, len_current_word):
  ''' Функция поиска слогов в слове '''

  for index_letter in range(1, len_current_word):

    if current_word[index_letter] in vowels and current_word[index_letter - 1] in consonants:  # Поиск слога в слове
      slog = current_word[index_letter - 1] + current_word[index_letter]

      find_index = DF_SLOG.loc[DF_SLOG.isin([slog]).any(axis=1)].index.tolist() # нахождение индекса в дата фрейме (номера строки) слога

      len_current_word_true = len_current_word
      if '-' in current_word:
        len_current_word_true = len_current_word - current_word.count('-') # поправка количества строк в слове
      if "'" in current_word:
        len_current_word_true -= current_word.count("'") # поправка количества строк в слове

      if len_current_word > 2 or ( current_word not in my_stop_words ):  # Если это слово от 3 букв и не входит в стоп слова
        DF_SLOG['Частота'][find_index[1]] += 1 # Увеличиваем число находжение слога в колонке частоты совпадений в Сокращенном тексте
        DF_SLOG[f'{len_current_word_true} буквы'][find_index[1]] += 1 # Увеличиваем число находжение слога в колонке длины слова где находится слог в Сокращенном тексте

      DF_SLOG['Частота'][find_index[0]] += 1 # Увеличиваем число находжение слога в колонке частоты совпадений в Полном тексте
      DF_SLOG[f'{len_current_word_true} буквы'][find_index[0]] += 1 # Увеличиваем число находжение слога в колонке длины слова где находится слог в Полном тексте


In [ ]:
%%time
for word_from_list_book in list_TEXT_BOOK:
  if len(word_from_list_book) > 1:
    word_processing(word_from_list_book, len(word_from_list_book))

In [ ]:
DF_SLOG

In [41]:
DF_SLOG.shape

(400, 26)

In [21]:
DF_SLOG_cleare = DF_SLOG.loc[DF_SLOG['Частота'] != 0] # отфильтруем ДатаФрей убрав слоги которые не встречаются
DF_SLOG_cleare.shape

(248, 26)

In [22]:
# Сохраняем ДатаФрейм для дальнейшей обработки, чтоб не выгружать его снова
DF_SLOG.to_csv('slog_sourse.csv', encoding="utf-8-sig")
DF_SLOG_cleare.to_csv('slog_cleare.csv', encoding="utf-8-sig")

### Этап 6: Анализ полученных данных

In [ ]:
!gdown --id 1erlFzbvkcHa5iyaLPX4XLW8mNrUaWhYO

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1erlFzbvkcHa5iyaLPX4XLW8mNrUaWhYO
To: /content/slog_cleare.csv
100% 33.2k/33.2k [00:00<00:00, 133MB/s]


In [ ]:
# Загрузка датаФрейма
DF_SLOG_work = pd.read_csv('/content/slog_cleare.csv')
DF_SLOG_work.drop(columns='Unnamed: 0', axis= 1 , inplace= True ) # Удаление стоб

In [ ]:
DF_SLOG_work

,Текст,Слог,Частота,2 буквы,3 буквы,4 буквы,5 буквы,6 буквы,7 буквы,8 буквы,...,19 буквы,20 буквы,21 буквы,22 буквы,23 буквы,24 буквы,25 буквы,26 буквы,27 буквы,28 буквы
0,Полный,ба,2613,2,109,158,225,361,499,595,...,2,4,0,0,0,0,0,0,0,0
1,Полный,бу,2769,1,18,155,1171,281,187,106,...,1,1,0,1,1,0,0,0,0,0
2,Полный,бо,5891,2,207,343,1358,1021,1073,566,...,3,4,2,1,0,1,0,0,0,0
3,Полный,бы,10885,1514,1589,5227,1087,668,385,165,...,0,0,0,0,0,0,0,1,0,0
4,Полный,би,1947,0,17,101,261,312,484,394,...,1,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Сокращенный,що,2,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
316,Сокращенный,щи,1983,0,3,60,109,187,372,238,...,2,3,2,0,0,1,0,0,0,0
317,Сокращенный,щэ,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318,Сокращенный,щя,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
DF_SLOG_work.groupby('Слог')['Частота'].mean()

Слог
ба    2613.0
бе    5682.0
би    1947.0
бо    5891.0
бу    2769.0
       ...  
щи    1983.0
що       2.0
щу     242.0
щэ       1.0
щя       1.0
Name: Частота, Length: 160, dtype: float64

In [ ]:
find_index = DF_SLOG.loc[DF_SLOG.isin(['вю']).any(axis=1)].index.tolist()

In [ ]:
find_index[1]

217

###Черновик

In [ ]:
list_x = [['ба'],
 ['бу'],
 ['бо'],
 ['бы'],
 ['би'],
 ['бэ'],
 ['бя'],
 ['бю'],
 ['бё'],
 ['бе'],
 ['ва'],
 ['ву'],
 ['во'],
 ['вы'],
 ['ви'],
 ['вэ'],
 ['вя'],
 ['вю'],
 ['вё'],
 ['ве']]

In [ ]:
list_x_double = list_x * 2

In [ ]:
list_x

[['ба'],
 ['бу'],
 ['бо'],
 ['бы'],
 ['би'],
 ['бэ'],
 ['бя'],
 ['бю'],
 ['бё'],
 ['бе'],
 ['ва'],
 ['ву'],
 ['во'],
 ['вы'],
 ['ви'],
 ['вэ'],
 ['вя'],
 ['вю'],
 ['вё'],
 ['ве']]

In [ ]:
list_x_double

[['ба'],
 ['бу'],
 ['бо'],
 ['бы'],
 ['би'],
 ['бэ'],
 ['бя'],
 ['бю'],
 ['бё'],
 ['бе'],
 ['ва'],
 ['ву'],
 ['во'],
 ['вы'],
 ['ви'],
 ['вэ'],
 ['вя'],
 ['вю'],
 ['вё'],
 ['ве'],
 ['ба'],
 ['бу'],
 ['бо'],
 ['бы'],
 ['би'],
 ['бэ'],
 ['бя'],
 ['бю'],
 ['бё'],
 ['бе'],
 ['ва'],
 ['ву'],
 ['во'],
 ['вы'],
 ['ви'],
 ['вэ'],
 ['вя'],
 ['вю'],
 ['вё'],
 ['ве']]

In [ ]:
len_x = len(list_x_double)

In [ ]:
len_x

40

In [ ]:
for d in range(len_x):
  if d < (len_x / 2 + 1):
    list_x_double[d].insert(0, 'Полный') # Дополняем вложенный список типом текста Первую половину слогов
    print(f'Добавляем ПОЛНЫЙ номер: {d}')
  else:
    list_x_double[d].insert(0, 'Сокращенный') # Дополняем вложенный список типом текста Вторую половину слогов
    print(f'Добавляем Сокращенный номер: {d}')
  list_x_double[d].append(0) # Дополняем вложенный список нулевыми значениями совпадений слогов
  print(f'присваеваем 0: {d}')

Добавляем ПОЛНЫЙ номер: 0
присваеваем 0: 0
Добавляем ПОЛНЫЙ номер: 1
присваеваем 0: 1
Добавляем ПОЛНЫЙ номер: 2
присваеваем 0: 2
Добавляем ПОЛНЫЙ номер: 3
присваеваем 0: 3
Добавляем ПОЛНЫЙ номер: 4
присваеваем 0: 4
Добавляем ПОЛНЫЙ номер: 5
присваеваем 0: 5
Добавляем ПОЛНЫЙ номер: 6
присваеваем 0: 6
Добавляем ПОЛНЫЙ номер: 7
присваеваем 0: 7
Добавляем ПОЛНЫЙ номер: 8
присваеваем 0: 8
Добавляем ПОЛНЫЙ номер: 9
присваеваем 0: 9
Добавляем ПОЛНЫЙ номер: 10
присваеваем 0: 10
Добавляем ПОЛНЫЙ номер: 11
присваеваем 0: 11
Добавляем ПОЛНЫЙ номер: 12
присваеваем 0: 12
Добавляем ПОЛНЫЙ номер: 13
присваеваем 0: 13
Добавляем ПОЛНЫЙ номер: 14
присваеваем 0: 14
Добавляем ПОЛНЫЙ номер: 15
присваеваем 0: 15
Добавляем ПОЛНЫЙ номер: 16
присваеваем 0: 16
Добавляем ПОЛНЫЙ номер: 17
присваеваем 0: 17
Добавляем ПОЛНЫЙ номер: 18
присваеваем 0: 18
Добавляем ПОЛНЫЙ номер: 19
присваеваем 0: 19
Добавляем ПОЛНЫЙ номер: 20
присваеваем 0: 20
Добавляем Сокращенный номер: 21
присваеваем 0: 21
Добавляем Сокращенный ном

In [ ]:
list_x_double

[['Полный', 'Полный', 'ба', 0, 0],
 ['Сокращенный', 'Полный', 'бу', 0, 0],
 ['Сокращенный', 'Полный', 'бо', 0, 0],
 ['Сокращенный', 'Полный', 'бы', 0, 0],
 ['Сокращенный', 'Полный', 'би', 0, 0],
 ['Сокращенный', 'Полный', 'бэ', 0, 0],
 ['Сокращенный', 'Полный', 'бя', 0, 0],
 ['Сокращенный', 'Полный', 'бю', 0, 0],
 ['Сокращенный', 'Полный', 'бё', 0, 0],
 ['Сокращенный', 'Полный', 'бе', 0, 0],
 ['Сокращенный', 'Полный', 'ва', 0, 0],
 ['Сокращенный', 'Полный', 'ву', 0, 0],
 ['Сокращенный', 'Полный', 'во', 0, 0],
 ['Сокращенный', 'Полный', 'вы', 0, 0],
 ['Сокращенный', 'Полный', 'ви', 0, 0],
 ['Сокращенный', 'Полный', 'вэ', 0, 0],
 ['Сокращенный', 'Полный', 'вя', 0, 0],
 ['Сокращенный', 'Полный', 'вю', 0, 0],
 ['Сокращенный', 'Полный', 'вё', 0, 0],
 ['Сокращенный', 'Полный', 'ве', 0, 0],
 ['Полный', 'Полный', 'ба', 0, 0],
 ['Сокращенный', 'Полный', 'бу', 0, 0],
 ['Сокращенный', 'Полный', 'бо', 0, 0],
 ['Сокращенный', 'Полный', 'бы', 0, 0],
 ['Сокращенный', 'Полный', 'би', 0, 0],
 ['Сокраще

In [ ]:
list_x.insert(0, 'sds')

In [ ]:
for s in range(2):
  print(s)

0
1


In [ ]:
g = 'ауоыиэяюёе' # гласные буквы
s = 'бвгджзклмнпрстфхцчшщ' # согласные буквы
TS = []

In [ ]:
for i in range(2):
  for ss in s:
    for gg in g:
      TS.append([ss + gg]) # Создали список с вложенным (2 уровня) списком слогов.

In [ ]:
TS